In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm
from user_info import get_user_data_search_embed
from job_info import fetch_jobs

In [2]:
# from vllm import SamplingParams, LLM

# sampling_params = SamplingParams(temperature=0, max_tokens=144)
# llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

In [3]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
#                                             load_in_8bit=True, 
#                                             torch_dtype=torch.float16,
#                                             device_map='auto')

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [4]:
dataset = pd.read_csv('data/analysis_dataset.csv')

In [5]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [6]:
def fetch_job_title(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    title = job_data[0]['title']
    jd = job_data[0]['introText']
    jd = html.unescape(jd)

    return title + '\n' + jd


In [7]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [8]:
dataset['job_id'].drop_duplicates()

0         1373681
25        1373682
66        1373683
71        1373684
189       1373685
           ...   
133468    1376446
133579    1376758
134063    1375146
136145    1376759
136172    1376737
Name: job_id, Length: 2949, dtype: int64

In [9]:
dataset['user_id'].drop_duplicates()

0          439042
1         2015063
2         2777927
3           58130
4         2290933
           ...   
136201    2796578
136202    2679806
136235     986712
136245    1713212
136249    1463255
Name: user_id, Length: 30363, dtype: int64

In [10]:
dataset.sample(5)

,job_id,application_id,user_id
82665,1375765,144345075,1299999
12238,1373862,144247897,1821067
125338,1374717,144422097,2742155
77059,1375508,144333416,1149207
101118,1376188,144378618,1220972


In [11]:
es_output = get_user_info(1358359)
rt = es_output['resume'][0]
rt = html.unescape(rt)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13751.82it/s]


In [12]:
answer_format = {
    "work_experience":"description of most recent work experience"
}

In [19]:
f = []
for jid in tqdm(dataset.job_id.unique()):
    sdf = fetch_jobs(str(jid))
    f.append(sdf)

jdf = pd.concat(f)

  0%|          | 0/2949 [00:00<?, ?it/s]

/tmp/ipykernel_23575/3556400985.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jdf = pd.concat(f)


In [20]:
jdf.sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,showRecruiterProfile,location
0,1374691,Deputy General Manager - CRM - Real Estate (10...,<p>KEY RESPONSIBILITIES:<br/><br/>Preferably 1...,Deputy General Manager - CRM,10,15,227980,thrivibe.com,14,https://www.iimjobs.com/j/deputy-general-manag...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Thrivibe - Th...","{'recruiterId': 227980, 'recruiterName': 'Ruch...",None,0,NaN,"[{'id': 7, 'name': 'Pune'}]"


In [16]:
jdf[jdf['applyCount']>=70].sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,showRecruiterProfile,location
0,1376606,Manager/Senior Manager - Risk & Compliance - K...,<p>JD :<br/><br/>- Risk & Compliance Managemen...,Manager/Senior Manager,6,10,157696,huquo.com,13,https://www.iimjobs.com/j/managersenior-manage...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Huquo', 'comp...","{'recruiterId': 157696, 'recruiterName': 'Kish...",None,1,NaN,"[{'id': 37, 'name': 'Gurgaon/Gurugram'}]"


In [20]:
#filter out jobs, select ones with atleast 50 applications
count_df = dataset['job_id'].value_counts().reset_index()
jobs = count_df.loc[(count_df['count']>=50) & (count_df['count']<60)].sample(10)

In [22]:
jobs

,job_id,count
840,1375026,51
848,1374956,50
752,1373879,56
851,1374981,50
798,1374473,54
796,1376033,54
729,1375642,58
835,1373929,51
845,1373732,51
852,1373755,50


In [26]:
sample_set = dataset[dataset['job_id'].isin(jobs['job_id'])]
job_details = sample_set[['job_id','user_id']]

In [52]:
applicants = job_details['user_id'].unique().tolist()

In [53]:
from user_info import fetch_data_es

In [54]:
es_data = fetch_data_es(applicants)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476/476 [00:03<00:00, 130.89it/s]


In [55]:
r[0][0]['id']

209995

In [56]:
resumes = {}

for data in tqdm(es_data):
    try:
        userid = data[0]['id']
        resumes[userid] = data[0]['resumeText']
    except:
        pass

  0%|          | 0/476 [00:00<?, ?it/s]

In [57]:
len(resumes.keys())

473

In [62]:
import random
random.sample(list(resumes.keys()),1)

[2808728]

In [65]:
import json
with open('data/test_user_resume.json','w') as f:
    json.dump(resumes,f)

In [66]:
import json
with open('data/test_user_resume.json','r') as f:
    check = json.load(f)

In [68]:
len(check.keys())

473